
# Metadata

**Topic** - unit test and debugging python

**Target Number of Turns (User + Assistant) - 3**

**Suggested Conversation :** - As a developer at a nonprofit organization focused on digital inclusivity, I've been tasked with ensuring that our web application is fully accessible to users with disabilities. Can you provide me with an example of how to use Python to write an automated test that checks for the presence of alt attributes on all images in our Django web app? Additionally, I would like the test to generate a report of the findings.

**User Personality :** - This user is practical, with a focused approach to problem-solving, and values collaboration over working independently. They demonstrate a commitment to real-world applications that have a direct impact on users.


# Conversation

**User**

I'm working on making our web application more accessible, and I need to ensure all our images have appropriate alt text for screen readers. Could you show me how to write a Python test for our Django app that checks for alt attributes on images and compiles a report on it?

**Assistant**

Absolutely! In Django, you can create a test case that inspects your HTML templates to ensure each **`<img>`** tag has an **`alt`** attribute. You would use Django's testing framework along with BeautifulSoup to parse the HTML. BeautifulSoup allows you to navigate and search the parse tree, which makes it easy to find all image tags and check for the presence of the **`alt`** attribute.

Let's start by writing a test case in Python:

```python
from django.test import TestCase, Client
from bs4 import BeautifulSoup
import timeout_decorator

class AccessibilityTestCase(TestCase):
    def setUp(self):
        # Setup the client to make requests.
        self.client = Client()

    @timeout_decorator.timeout(30)  # Add a 30-second timeout.
    def test_image_alt_tags(self):
        # Define a list to hold any images missing alt tags.
        images_missing_alt = []

        # Get the response from your Django app.
        response = self.client.get('/your-page-url/')

        # Parse the response content using BeautifulSoup.
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all image tags.
        images = soup.find_all('img')

        # Check each image for an alt attribute.
        for img in images:
            if not img.has_attr('alt') or not img['alt'].strip():
                images_missing_alt.append(img)

        # Assert that all images have alt attributes.
        error_message = f"Images missing alt tags on {url}: {images_missing_alt}"
        self.assertTrue(len(images_missing_alt) == 0, error_message)

        # Optionally, generate a report.
        if images_missing_alt:
            with open('missing_alt_report.txt', 'w') as report:
                for img in images_missing_alt:
                    report.write(f"Image with src '{img['src']}' is missing an alt tag on {url}.\n")
```

The code provided should be saved in a test file within your Django app's tests directory. For example, if your app is named **`myapp`**, you would save the code in **`myapp/tests/test_accessibility.py`**.

1. To run the test using Django's test runner, you would use the following command in your terminal:

  ```bash
    python manage.py test myapp
  ```
  
  This will run all tests in the `myapp` app.

2. If you want to run a specific test, you can specify the test module and class:

  ```bash
    python manage.py test myapp.tests.test_accessibility.AccessibilityTestCase
  ```

Here is a summary of the code:
* **`AccessibilityTestCase`** is a test case class that checks if all images on a given page have alt tags.

* **`setUp`** creates a **`Client`** instance for making requests to the Django app.

* A timeout is added to prevent the test from hanging indefinitely using **`timeout_decorator`**.

* **`test_image_alt_tags`** makes a GET request to the specified URL, parses the response content using **`BeautifulSoup`**, finds all **`img`** tags, and checks if they have **`alt`** attributes.

* If any images are missing alt tags, an error message is generated and written to a text file.

* This test case helps ensure that the Django app meets basic accessibility standards for images.

This test case checks for accessibility issues related to images on a web page by ensuring that all images have alt tags. Alt tags are important for accessibility because they provide a text description of an image for users who are visually impaired or use screen readers. Make sure to replace **`myapp`** with the actual name of your Django app. Also, don't forget to replace **`'/your-page-url/'`** with the actual URL you want to test.

**User**

How can I adjust the testing strategy to ensure comprehensive coverage across all pages of our Django application, beyond just focusing on individual URLs?

**Assistant**

To check the entire Django application, you can modify the test to recursively visit all URLs defined in your URL configuration.

Here's how you can approach it:

```python
from django.urls import reverse
from django.core.urlresolvers import resolve
from django.test import TestCase, Client
from bs4 import BeautifulSoup

class AccessibilityTestCase(TestCase):
    def setUp(self):
        self.client = Client()
        self.visited_urls = set()
        self.images_missing_alt = []

    def test_site_for_image_alt_tags(self):
        # Get all urls from your Django application.
        urlconf = __import__(settings.ROOT_URLCONF, {}, {}, [''])
        self.visit_urls(urlconf.urlpatterns)

        # Assert no missing alt tags across the site.
        self.assertTrue(len(self.images_missing_alt) == 0, f"Images missing alt tags: {self.images_missing_alt}")

        # Generate a report.
        if self.images_missing_alt:
            with open('missing_alt_report.txt', 'w') as report:
                for img in self.images_missing_alt:
                    report.write(f"Image with src '{img['src']}' is missing an alt tag on page {img['page']}.\n")

    def visit_urls(self, urlpatterns, prefix=''):
        for pattern in urlpatterns:
            if hasattr(pattern, 'url_patterns'):
                # This is an included URLconf, recurse into it.
                self.visit_urls(pattern.url_patterns, prefix=prefix + pattern.regex.pattern)
            else:
                # This is a final URL pattern, add it if we have not visited it yet.
                url = prefix + pattern.regex.pattern
                if url not in self.visited_urls:
                    self.visited_urls.add(url)
                    self.check_url_for_alt_tags(url)

    def check_url_for_alt_tags(self, url):
        try:
            response = self.client.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            images = soup.find_all('img')
            for img in images:
                if not img.has_attr('alt') or img['alt'].strip() == '':
                    self.images_missing_alt.append({'src': img.get('src'), 'page': url})
        except:
            pass  # Handle exceptions for URLs that require a specific setup (e.g., login).
```

In this code:

- The **`test_site_for_image_alt_tags`** method is the entry point of the test, initiating the crawling process.

- The **`visit_urls`** method recursively visits all URLs in your Django app's URL configuration.

- For each URL, **`check_url_for_alt_tags`** is called to perform the actual check for missing **`alt`** tags.

- Images missing **`alt`** tags are collected into **`self.images_missing_alt`** with the source (**`src`**) of the image and the page the image was found on.

- The test will assert that no images are missing **`alt`** tags.

- A report is generated listing all images without **`alt`** attributes, including the page where each image is located.

It's important to note that this approach may not cover all possible scenarios, such as **dynamic URLs**, URLs that require specific conditions or data, and URLs that are not accessible via **GET** requests. In such cases, you may need to customize the **`visit_urls`** method or use additional testing strategies to ensure comprehensive coverage. Please ensure you have the necessary error handling and consider the implications of crawling your live application, as this could potentially generate a significant number of requests. It's often best to run such tests in a controlled staging environment.

**User**

Could you also add a feature to this test that checks for overly long alt text? Some of our contributors tend to write very descriptive alt text, which might not be ideal for screen readers.

**Assistant**

Certainly! I can add an additional check to the test to flag any **`alt`** attributes that are excessively long. A common guideline is to keep alt text under **125 characters**, so we can use this as a threshold.

Here's the updated code with the new feature:

```python
from django.test import TestCase, Client
from bs4 import BeautifulSoup

class AccessibilityTestCase(TestCase):
    def setUp(self):
        # Setup the client to make requests.
        self.client = Client()

    def test_image_alt_tags(self):
        # Lists to hold images with issues.
        images_missing_alt = []
        images_with_long_alt = []

        # Get the response from your Django app.
        response = self.client.get('/your-page-url/')  # Replace with your actual URL.

        # Parse the response content using BeautifulSoup.
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all image tags.
        images = soup.find_all('img')

        # Check each image for an alt attribute and length.
        for img in images:
            alt_text = img.get('alt', '')
            if not alt_text or alt_text.strip() == '':
                images_missing_alt.append(img['src'])
            elif len(alt_text) > 125:
                images_with_long_alt.append((img['src'], alt_text))

        # Assert that all images have alt attributes and they are not too long.
        self.assertTrue(len(images_missing_alt) == 0, f"Images missing alt tags: {images_missing_alt}")
        self.assertTrue(len(images_with_long_alt) == 0, f"Images with overly long alt text: {images_with_long_alt}")

        # Generate a report if there are issues.
        if images_missing_alt or images_with_long_alt:
            with open('alt_text_issues_report.txt', 'w') as report:
                if images_missing_alt:
                    report.write("Images missing alt tags:\n")
                    for src in images_missing_alt:
                        report.write(f"{src}\n")
                if images_with_long_alt:
                    report.write("\nImages with overly long alt text:\n")
                    for src, alt in images_with_long_alt:
                        report.write(f"{src} has alt text of {len(alt)} characters\n")
```

In this code:

- Added a check for the length of the **`alt`** attribute text.

- If an image's **`alt`** text is longer than 125 characters, it gets added to the **`images_with_long_alt`** list.

- The test will fail if there are any images with missing **`alt`** attributes or overly long **`alt`** text.

- The report now includes sections for both types of issues: missing **`alt`** attributes and overly long **`alt`** text.

- Each section of the report lists the relevant image **`src`** attributes and, for long **`alt`** text, the length of the **`alt`** text.

In conclusion, the code snippet demonstrates how to check the length of alt attributes and flag any that are excessively long. However, it's important to consider the specific requirements and constraints of your Django application and adjust the threshold for **`alt`** text length accordingly.